In [1]:
import numpy as np
import pandas as pd
import os
import unittest
from IPython.display import display

In [2]:
# Load environmental variables

# Assign raw data and plate diagram files from environment
RAW_SAMPLE_1 = "./data/test/raw_counts.xlsx"
RAW_SAMPLE_1_DIAGRAM = "./data/test/loading_diagram.xlsx"

# Read in the raw data and plate diagram as pandas dataframes
df_data = pd.read_excel(RAW_SAMPLE_1, sheet_name=0)
df_diagram = pd.read_excel(RAW_SAMPLE_1_DIAGRAM)


# Test the raw data and plate diagram to make sure they are the correct size, should be 96 rows by 16 columns and 8 rows by 13 columns respectively always
class TestStringMethods(unittest.TestCase):

    def test_raw_data(self):
        self.assertEqual(df_data.shape, (96, 16), f"Raw data is {df_data.shape[0]} rows by {df_data.shape[1]} columns but should be 96 rows by 3 columns")

    def test_plate_diagram(self):
        self.assertEqual(df_diagram.shape, (8, 13), f"Plate diagram is {df_diagram.shape[0]} rows by {df_diagram.shape[1]} columns but should be 8 rows by 13 columns")


unittest.main(argv=['first-arg-is-ignored'], exit=False)

display(df_data)
display(df_diagram)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


,Unnamed: 0,Well,Fluor,Target,Content,Sample,Biological Set Name,Cq,Cq Mean,Cq Std. Dev,Starting Quantity (SQ),Log Starting Quantity,SQ Mean,SQ Std. Dev,Set Point,Well Note
0,NaN,A01,SYBR,NaN,Unkn,NaN,NaN,NaN,0.000000,0,NaN,NaN,0.0,0,72,NaN
1,NaN,A02,SYBR,NaN,Unkn,NaN,NaN,20.971671,20.971671,0,NaN,NaN,NaN,0,72,NaN
2,NaN,A03,SYBR,NaN,Unkn,NaN,NaN,20.832234,20.832234,0,NaN,NaN,NaN,0,72,NaN
3,NaN,A04,SYBR,NaN,Unkn,NaN,NaN,20.146131,20.146131,0,NaN,NaN,NaN,0,72,NaN
4,NaN,A05,SYBR,NaN,Unkn,NaN,NaN,20.245704,20.245704,0,NaN,NaN,NaN,0,72,NaN


,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12
0,A,NaN,D99,D99 dup,D119,D119 dup,D143,D143 dup,D162,D162 dup,D191,D191 dup,NaN
1,B,NaN,D103,D103 dup,D122,D122 dup,D146,D146 dup,D163,D163 dup,D194,D194 dup,NaN
2,C,NaN,D106,D106 dup,D130,D130 dup,D150,D150 dup,D171,D171 dup,D198,D198 dup,NaN
3,D,NaN,D110,D110 dup,D131,D131 dup,D151,D151 dup,D174,D174 dup,D199,D199 dup,NaN
4,E,NaN,D111,D111 dup,D135,D135 dup,D154,D154 dup,D175,D175 dup,D131 SCR,D131 SCR dup,NaN


In [16]:
# Set the first column as the index, remove whitespace and add a space to the "dup" values
df_diagram = df_diagram.set_index(df_diagram.columns[0])
df_diagram = df_diagram.replace('\s+', '', regex=True)
df_diagram = df_diagram.replace('dup', ' dup', regex=True)

# Create an empty sample map dictionary
sample_map = {}

for row in df_diagram.index:
    for col in df_diagram.columns[1:]:
        well_id = f"{row}{int(col):02d}"
        sample_name = df_diagram.loc[row, col]
        sample_map[well_id] = sample_name

    """Use rows and columns (besides the first one) to relate the well ID to the sample name.

    For example, the first row name is A and the first column name is 1.
    The well ID will be A01 and the sample name will the sample in that row/column.

    """

# Read in the raw qPCR data and map the well IDs to sample names using the dictionary
df_data["Sample"] = df_data["Well"].map(sample_map)

# Show first 5 rows
df_data.head()

,Unnamed: 0,Well,Fluor,Target,Content,Sample,Biological Set Name,Cq,Cq Mean,Cq Std. Dev,Starting Quantity (SQ),Log Starting Quantity,SQ Mean,SQ Std. Dev,Set Point,Well Note
0,NaN,A01,SYBR,NaN,Unkn,NaN,NaN,NaN,0.000000,0,NaN,NaN,0.0,0,72,NaN
1,NaN,A02,SYBR,NaN,Unkn,NaN,NaN,19.214765,19.214765,0,NaN,NaN,NaN,0,72,NaN
2,NaN,A03,SYBR,NaN,Unkn,NaN,NaN,19.586924,19.586924,0,NaN,NaN,NaN,0,72,NaN
3,NaN,A04,SYBR,NaN,Unkn,NaN,NaN,19.634851,19.634851,0,NaN,NaN,NaN,0,72,NaN
4,NaN,A05,SYBR,NaN,Unkn,NaN,NaN,19.645373,19.645373,0,NaN,NaN,NaN,0,72,NaN


In [17]:
# Select relevant columns (Well, Cq, and Sample)
df = df_data[['Well', 'Cq', 'Sample']].copy()

# Create mtDNA1 and mtDNA2 columns
df['mtDNA1'] = "mtDNA1"
df['mtDNA2'] = "mtDNA2"

# Arrange columns like this: "Well", "Sample", "mtDNA1", "mtDNA2", "Cq"
df = df.loc[:,["Well", "Sample", "mtDNA1", "mtDNA2", "Cq"]]

# Drop rows with NA values
df = df.dropna()

# Show first 5 rows
df.head()

,Well,Sample,mtDNA1,mtDNA2,Cq


In [18]:
# set mtDNA1 and mtDNA2 values to Cq values by treating mtDNA1 as the Cq for the first sample and mtDNA2 as the Cq for the duplicate sample if it exists as "Sample dup"

# Note, exactly "Sample dup" is used to avoid matching "Sample dup **" or any additions to the name

for row, index in df.iterrows():
    df.loc[row, 'mtDNA1'] = df.loc[row, 'Cq']
    if df.loc[row, 'Sample'] + ' dup' in df['Sample'].values:
        df.loc[row, 'mtDNA2'] = df.loc[df['Sample'] == df.loc[row, 'Sample'] + ' dup', 'Cq'].values[0]
    else:
        df.loc[row, 'mtDNA2'] = np.NAN

""" Assign mtDNA1 and mtNDA2 value

    For each row, set the mtDNA1 value to the Cq value and set the mtDNA2 value to the Cq value of the duplicate sample if it exists.

    For example, if the sample name is "D12", set the mtDNA1 value to its Cq value and set the mtDNA2 value to the Cq value of "D12 Dup" (if it exists).

    If the duplicate ("D12 Dup") does not exist, set the mtDNA2 value to NaN, which is then dropped later.
"""


# Drop the Cq column and drop NA values
df = df.drop(columns=['Cq'])
df = df.dropna()

# calculate standard deviation of each row
df['St.Dev'] = df[['mtDNA1', 'mtDNA2']].std(axis=1)

# Show first 5 rows
df.head()

,Well,Sample,mtDNA1,mtDNA2,St.Dev


In [13]:
# Throw warnings for standard deviations greater than .22

for row, index in df.iterrows():
    if df.loc[row, 'St.Dev'] > .22:
        print(f"\n Warning: Standard deviation for {df.loc[row, 'Sample']} is {round(df.loc[row, 'St.Dev'],ndigits=3)} "
              f"(Sample 1: {round(df.loc[row, 'mtDNA1'],ndigits=3)} vs Sample 2: {round(df.loc[row, 'mtDNA2'], ndigits=2)}) \n")

In [28]:
#Drop index, sort by standard deviation (descending), and download the file
df = df.sort_values(by=['St.Dev'], ascending=False)
df = df.reset_index(drop=True)

df.to_excel("50_gcr_random_name_test_output.xlsx",
          index=False)